In [ ]:
import pandas as pd
from openpyxl import load_workbook
from scipy import stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy

In [ ]:
### import data from excel workbook

wb = load_workbook('../../../Documents/Nanostring/Records/Initial Dataset.xlsx')

print(wb.sheetnames)
ws = wb['Exported dataset']

values = [[y.value for y in x] for x in ws[ws.calculate_dimension()]]
# values

In [ ]:
### Convert nested list to a pandas dataFrame and extract expression data with labels

df = pd.DataFrame(values)

rowLabels = df.iloc[23:,3]
# rowLabels = [x.split(' (')[0] for x in rowLabels.values]
rowLabels = dict(zip([x for x in range(23,23+len(rowLabels))],rowLabels))
rowLabels

colLabels = df.iloc[0,4:]
colLabels = [x.replace(' | ','_') for x in colLabels.values]
colLabels = dict(zip([x for x in range(4,4+len(colLabels))],colLabels))
colLabels

data = df.iloc[23:,4:].astype(np.float32)
data.rename(index=rowLabels,columns=colLabels, inplace=True)


data

dataOrig = data



In [ ]:
data.loc['HYB-POS']

posGMean = stats.gmean([round(x) for x in data.loc['HYB-POS'].values])


In [ ]:
posGMean

In [ ]:
data = (data/data.loc['HYB-POS'])*posGMean

In [ ]:
data

In [ ]:
plt.matshow(data)

In [ ]:
mean = data.mean(axis = 1)
data = data.assign(mean=mean.values)

In [ ]:
mean = data.mean(axis = 1)
data = data.assign(mean=mean.values)

probeClass = df.iloc[23:,2]      ### Index needs updating here also

probeClass.rename(index=rowLabels, inplace=True)
probeClass.rename(index='ProbeClass', inplace=True)
probeClass

probeClassDict = {
    'Positive': 'A',
    'Negative': 'B',
    'Control': 'C',
    'Endogenous': 'E'
}
data = data.assign(probeClass=[probeClassDict[v] for v in probeClass.values])

In [ ]:
probeClass.shape

In [ ]:
probeClassDict = {
    'Positive': 'A',
    'Negative': 'B',
    'Control': 'C',
    'Endogenous': 'E'
}
data = data.assign(probeClass=[probeClassDict[v] for v in probeClass.values])

In [ ]:

dataSorted = data.sort_values(by = ['probeClass', 'mean'], ascending=[True,True])# , inplace=True)  #, kind='quicksort', na_position='last', ignore_index=False, key=None)




In [ ]:
### Extract control and endogenous labels for targets/probes

TargetClass = pd.DataFrame(df.iloc[28:,3:4])
TargetClass.rename(index=df.iloc[28:,2], columns=df.iloc[27,2:4], inplace=True)

negCTLs = TargetClass.loc['Negative'].values.flatten()
posCTLs = TargetClass.loc['Positive'].values.flatten()
HK = TargetClass.loc['Control'].values.flatten()
endog = TargetClass.loc['Endogenous'].values.flatten()

print(negCTLs.shape)
print(posCTLs.shape)
print(HK.shape)
print(endog.shape)



In [ ]:
negCTLs = [negCTLs[0]] + [negCTLs[2]]

In [ ]:
HK= [HK[0]] + [HK[2]]

In [ ]:
# To drop mean and probeClass columns use the following code
# data.drop(labels=['mean','probeClass'], axis=1)

# Calculate negative control Geometric mean and Housekeeping gene Geometric mean
negGMean = stats.gmean([[round(x) for x in y ] for y in data.drop(labels=['mean','probeClass'], axis=1).loc[negCTLs].values])
HKGMean = stats.gmean([[round(x) for x in y ] for y in data.drop(labels=['mean','probeClass'], axis=1).loc[HK].values])

negGMean = pd.Series(negGMean, index = data.columns[:-2])
HKGMean = pd.Series(HKGMean, index = data.columns[:-2])
negGMeanLog2 = negGMean.transform(lambda x : np.log2(x))
HKGMeanLog2 = HKGMean.transform(lambda x : np.log2(x))
# negGMeanLog2.sort_values(inplace=True)
# HKGMeanLog2.sort_values(inplace=True)

In [ ]:
negGMean

In [ ]:
HKGMeanLog2

In [ ]:
endogData = data.drop(labels=['mean','probeClass'], axis=1).loc[endog]
# endogNorm = endogData/HKGMean
endogNorm = endogData/negGMean

In [ ]:
endogNorm

In [ ]:
### Extract descriptions for each sample

sampleInfo = pd.DataFrame(df.iloc[0:17,4:])
sampleInfo.rename(index=df.iloc[0:17,0], columns=colLabels, inplace=True)

print(sampleInfo.shape)
sampleInfo


In [ ]:
nuclei = sampleInfo.loc['AOI nuclei count']#.values.flatten()
surfArea = sampleInfo.loc['AOI surface area']#.values.flatten()


In [ ]:
negScaled = (data.drop(labels=['mean','probeClass'], axis=1)/negGMean).applymap(np.log2)


mean = negScaled.mean(axis = 1)
negScaled = negScaled.assign(mean=mean.values)



negScaled = negScaled.assign(probeClass=[probeClassDict[v] for v in probeClass.values])


negScaled.sort_values(by = ['probeClass', 'mean'], ascending=[True,True] , inplace=True)

In [ ]:
negScaled

In [ ]:
data

In [ ]:
# Log2 Signal-to-Background Ratio
plt.figure(figsize=(20,10))

plt.boxplot(negScaled.drop(labels=['mean','probeClass'], axis=1).T, sym='.', labels=negScaled.index)
plt.violinplot(negScaled.drop(labels=['mean','probeClass'], axis=1).T)
plt.xticks(rotation = 90)
plt.axhline(y=0.0, color='r', linestyle='-')


In [ ]:
# colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' for x in data.drop(labels=['mean','probeClass'], axis=1).columns]
colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' if x.split('_')[-1] == 'Immune' else 'purple' for x in data.drop(labels=['mean','probeClass'], axis=1).columns]



In [ ]:
plt.figure(figsize=(10,5))
plt.hist(np.log2(HKGMean), bins=12)
plt.xlim(0,15)

In [ ]:
HKGMeanLog2 = HKGMeanLog2.sort_values()

colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' if x.split('_')[-1] == 'Immune' else 'purple' for x in HKGMeanLog2.index]

plt.figure(figsize=(10,5))
plt.bar(np.linspace(1,len(HKGMeanLog2),len(HKGMeanLog2)), HKGMeanLog2, color=colours)
plt.ylim(0,15)

### Need to colour the bars by some defining features

In [ ]:
# Frequency vs AOIs log2 HK Geomean(Counts)
# AOIs log2 HK Geomean(Counts) vs Individual AOIs

# Frequency vs AOIs log2 IgG Geomean(Counts)
# AOIs log2 IgG Geomean(Counts) vs Individual AOIs

plt.figure(figsize=(10,5))

plt.hist(np.log2(negGMean), bins=12)
plt.xlim(0,15)



In [ ]:
###
plt.figure(figsize=(10,5))
negGMeanLog2 = negGMeanLog2.sort_values()
# colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' for x in negGMeanLog2.index]
colours = ['g' if x.split('_')[-1] == 'Tumour' else 'r' if x.split('_')[-1] == 'Immune' else 'purple' for x in negGMeanLog2.index]

plt.bar(np.linspace(1,len(negGMeanLog2),len(negGMeanLog2)), negGMeanLog2, color=colours)
# plt.semilogy()
plt.ylim(0,15)


In [ ]:
###
# Matrix of HK expression plots
rows = 3
cols = 3

fig, axes = plt.subplots(rows,cols, figsize=(15,15))


S6 = [np.log2(x) for x in data.drop(labels=['mean','probeClass'], axis=1).loc['S6'].values]
H3 = [np.log2(x) for x in data.drop(labels=['mean','probeClass'], axis=1).loc['Histone H3'].values]
GAPDH = [np.log2(x) for x in data.drop(labels=['mean','probeClass'], axis=1).loc['GAPDH'].values]

labels = ['S6', 'H3', 'GAPDH']
dataList = [S6, H3, GAPDH]

axMin = int(min([min(x) for x in dataList]))
axMax = int(max([max(x) for x in dataList]))+1


for r in range(rows):
    for c in range(cols):
        if r==c: # This is a diagonal, insert a label
            axes[r][c].text(0.5,0.5, labels[r],ha='center', va='center', transform=axes[r][c].transAxes,fontsize=36)
        else:
            axes[r][c].scatter(dataList[c], dataList[r], c=colours)
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(dataList[c], dataList[r])
            axes[r][c].text(0.9,0.9, "r_value = " + str(r_value)[:5],ha='right', va='center', transform=axes[r][c].transAxes)
            axes[r][c].text(0.9,0.8, "slope = " + str(slope)[:5],ha='right', va='center', transform=axes[r][c].transAxes)
            axes[r][c].set_xlim(axMin,axMax)
            axes[r][c].set_ylim(axMin,axMax)


In [ ]:

# Matrix of IgG expression plots
rows = 3
cols = 3

fig, axes = plt.subplots(rows,cols, figsize=(15,15))


mIG1 = [np.log2(x) for x in data.drop(labels=['mean','probeClass'], axis=1).loc['Ms IgG1'].values]
mIG2 = [np.log2(x) for x in data.drop(labels=['mean','probeClass'], axis=1).loc['Ms IgG2a'].values]
rIG = [np.log2(x) for x in data.drop(labels=['mean','probeClass'], axis=1).loc['Rb IgG'].values]

labels = ['Ms_IgG1', 'Ms_IgG2a', 'Rb_IgG']
dataList = [mIG1,mIG2,rIG]

axMin = int(min([min(x) for x in dataList]))
axMax = int(max([max(x) for x in dataList]))+1


for r in range(rows):
    for c in range(cols):
        if r==c: # This is a diagonal, insert a label
            axes[r][c].text(0.5,0.5, labels[r],ha='center', va='center', transform=axes[r][c].transAxes,fontsize=36)
        else:
            axes[r][c].scatter(dataList[c], dataList[r], c=colours)
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(dataList[c], dataList[r])
            axes[r][c].text(0.9,0.9, "r_value = " + str(r_value)[:5],ha='right', va='center', transform=axes[r][c].transAxes)
            axes[r][c].text(0.9,0.8, "slope = " + str(slope)[:5],ha='right', va='center', transform=axes[r][c].transAxes)
            axes[r][c].set_xlim(axMin,axMax)
            axes[r][c].set_ylim(axMin,axMax)
                

In [ ]:
# Matrix of IgG expression plots
rows = 4
cols = 4

fig, axes = plt.subplots(rows,cols, figsize=(15,15))

labels = ['HKGMean', 'negGMean', 'Area', 'Nuclei']
dataList = [np.log2(HKGMean), np.log2(negGMean), np.log2(list(surfArea.values)), np.log2(list(nuclei.values))]

axMin = int(min([min(x) for x in dataList]))
axMax = int(max([max(x) for x in dataList]))+1


for r in range(rows):
    for c in range(cols):
        if r==c: # This is a diagonal, insert a label
            axes[r][c].text(0.5,0.5, labels[r],ha='center', va='center', transform=axes[r][c].transAxes,fontsize=36)
        else:
            axes[r][c].scatter(dataList[c], dataList[r], c=colours)
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(dataList[c], dataList[r])
            axes[r][c].text(0.9,0.9, "r_value = " + str(r_value)[:5],ha='right', va='center', transform=axes[r][c].transAxes)
            axes[r][c].text(0.9,0.8, "slope = " + str(slope)[:5],ha='right', va='center', transform=axes[r][c].transAxes)
            axes[r][c].set_xlim(axMin,axMax)
            axes[r][c].set_ylim(axMin,axMax)
                
            
        
    

In [ ]:
dataHKGMScaled = data.drop(labels=['mean','probeClass'], axis=1).reindex(labels=negScaled.index)/HKGMean
dataHKGMScaled.to_csv('HK_Geo_Mean_Scaled.csv')

dataNegGMScaled = data.drop(labels=['mean','probeClass'], axis=1).reindex(labels=negScaled.index)/negGMean
dataNegGMScaled.to_csv('Neg_Geo_Mean_Scaled.csv')

In [ ]:
# Plot raw data before any normalisation


plt.figure(figsize=(20,10))

plt.boxplot(dataOrig.reindex(labels=negScaled.index).T.applymap(np.log2), sym='-', labels=negScaled.index)

for i,j in enumerate(negScaled.index):
    y = dataOrig.loc[j]
    colours = ['g.' if v.split('_')[-1] == 'Tumour' else 'r.' if v.split('_')[-1] == 'Immune' else 'b.' for v in y.index]
    y = y
    y = y.values
    x = np.random.normal(i+1, 0.1, len(y))
    
    for i in range(len(x)): 
        plt.plot(x[i], np.log2(y[i]), colours[i])
plt.xticks(rotation = 90)
plt.xlabel=list(negScaled.index)




In [ ]:
# plot RCC normalised data

plt.figure(figsize=(20,10))

plt.boxplot(data.drop(labels=['mean','probeClass'], axis=1).reindex(labels=negScaled.index).T.applymap(np.log2), sym='-', labels=negScaled.index)

for i,j in enumerate(negScaled.index):
    y = data.loc[j].drop(index=['mean','probeClass'], axis=1)
    colours = ['g.' if v.split('_')[-1] == 'Tumour' else 'r.' if v.split('_')[-1] == 'Immune' else 'b.' for v in y.index]
    y = y
    y = y.values
    x = np.random.normal(i+1, 0.1, len(y))
    for i in range(len(x)): 
        plt.plot(x[i], np.log2(y[i]), colours[i])
plt.xticks(rotation = 90)
plt.xlabel=list(negScaled.index)




In [ ]:
# Plot HK normalised data
plt.figure(figsize=(20,10))

plt.boxplot((data.drop(labels=['mean','probeClass'], axis=1).reindex(labels=negScaled.index)/HKGMean).T.applymap(np.log2), sym='-', labels=negScaled.index)

for i,j in enumerate(negScaled.index):
    y = data.loc[j].drop(index=['mean','probeClass'], axis=1)
    colours = ['g.' if v.split('_')[-1] == 'Tumour' else 'r.' if v.split('_')[-1] == 'Immune' else 'b.' for v in y.index]
    y = y/HKGMean
    y = y.values
    x = np.random.normal(i+1, 0.1, len(y))
    
    for i in range(len(x)): 
        plt.plot(x[i], np.log2(y[i]), colours[i])#, alpha=0.02)
plt.xticks(rotation = 90)
    


In [ ]:
# Plot negative probe normalised data
plt.figure(figsize=(20,10))

plt.boxplot(negScaled.drop(labels=['mean','probeClass'], axis=1).T, sym='.', labels=negScaled.index)

for i,j in enumerate(negScaled.index):
    y = data.loc[j].drop(index=['mean','probeClass'], axis=1)
    colours = ['g.' if v.split('_')[-1] == 'Tumour' else 'r.' if v.split('_')[-1] == 'Immune' else 'b.' for v in y.index]
    y = y/negGMean
    y = y.values
    x = np.random.normal(i+1, 0.1, len(y))
    
    for i in range(len(x)): 
        plt.plot(x[i], np.log2(y[i]), colours[i])
plt.xticks(rotation = 90)
plt.axhline(y=0.0, color='r', linestyle='-')
plt.xlabel=list(negScaled.index)
    


In [ ]:
# To Do:

### Generate new dataFrame with mask of where individual values are below HYB-NEG


### Generate new dataFrame with mask of where individual values are below NegGeoMean 
### Generate new dataFrame with mask of where mean value of a probe is below NegGeoMean 